<a href="https://colab.research.google.com/github/Murad1997/DS-Portfolio/blob/main/Data_analysis_and_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Loading

In [3]:
import pandas as pd
import matplotlib.pyplot as plt

# Exploratory Data Analysis

# Modeling

# Conclusion